# Worksheet 07 - JSON

Your Name:  
Your Class:   INST 447  
Your Section: 0101 (MWF) or 0102 (TTh)  
Your favorite color:  

In [1]:
import pandas as pd
import numpy as np
import json

I have saved the JSON response for each geocode request to the Google API for each Capital Area Bikeshare bike station into a csv file called "cabi_with_geo.csv".

Load the CSV into a Pandas DataFrame called 'stations'. Then parse out the JSON response into a two new columns - one for the latitude and one for the longitude.

Remove the column with the json response in it, so that you have a DataFrame that has the bikestation number, the bikestation address, the bikestation latitude, and the bikestation longitude.

The latitude and longitude are within geometry/location in the JSON.

In [2]:
stations = pd.read_csv('data/cabi_with_geo.csv')

In [3]:
stations.head()

,Unnamed: 0,number,address,response
0,0,31111.0,10th & U St NW,"{\n ""results"" : [\n {\n ""addres..."
1,1,31602.0,Park Rd & Holmead Pl NW,"{\n ""results"" : [\n {\n ""addres..."
2,2,31106.0,Calvert St & Woodley Pl NW,"{\n ""results"" : [\n {\n ""addres..."
3,3,31110.0,20th St & Florida Ave NW,"{\n ""results"" : [\n {\n ""addres..."
4,4,31107.0,Lamont & Mt Pleasant NW,"{\n ""results"" : [\n {\n ""addres..."


In [4]:
stations.drop(['Unnamed: 0'], axis='columns', inplace=True)

In [5]:
stations['number'] = stations['number'].astype('int64')

In [6]:
stations.head()

,number,address,response
0,31111,10th & U St NW,"{\n ""results"" : [\n {\n ""addres..."
1,31602,Park Rd & Holmead Pl NW,"{\n ""results"" : [\n {\n ""addres..."
2,31106,Calvert St & Woodley Pl NW,"{\n ""results"" : [\n {\n ""addres..."
3,31110,20th St & Florida Ave NW,"{\n ""results"" : [\n {\n ""addres..."
4,31107,Lamont & Mt Pleasant NW,"{\n ""results"" : [\n {\n ""addres..."


In [7]:
jd = json.loads(stations['response'][0])

In [8]:
lat = jd['results'][0]['geometry']['location']['lat']
lng = jd['results'][0]['geometry']['location']['lng']

In [9]:
geo_list = []
for lines in stations['response']:
    jd = json.loads(lines)
    if jd['status'] == 'OK':
        lat = jd['results'][0]['geometry']['location']['lat']
        lng = jd['results'][0]['geometry']['location']['lng']
    else:
        (lat, lng) = (np.NaN, np.NaN)
    geo_list.append({'lat': lat, 'lng': lng})

In [10]:
geos = pd.DataFrame(geo_list)

In [11]:
stations = pd.concat([stations, geos], axis=1)

In [16]:
stations.head()

,number,address,lat,lng
0,31111,10th & U St NW,38.917007,-77.025986
1,31602,Park Rd & Holmead Pl NW,38.930927,-77.030867
2,31106,Calvert St & Woodley Pl NW,38.923468,-77.050365
3,31110,20th St & Florida Ave NW,38.915107,-77.044913
4,31107,Lamont & Mt Pleasant NW,38.931139,-77.038249


In [14]:
stations = stations.dropna()
stations.drop(['response'], axis='columns', inplace=True)

In [15]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 0 to 106
Data columns (total 4 columns):
number     106 non-null int64
address    106 non-null object
lat        106 non-null float64
lng        106 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 4.1+ KB


If we want to map the the start and end of each bike trip, then we need to merge that stations' latitude and longitude with the trip data. Load the trips data from the "2010-q4-cabi-split-stations-columns.csv" into a DataFrame called trips. (This is the CABI data, but after I split the Start station and End station columns.)

Now add columns with the latitude and longitude for the start station.

In [17]:
trips = pd.read_csv('data/2010-q4-cabi-split-stations-columns.csv')

In [25]:
trips = trips.dropna()

In [26]:
trips['start_station_number'] = trips['start_station_number'].astype('int64')

In [27]:
trips.head()

,Unnamed: 0,Duration,Start date,End date,Start station,End station,Bike#,Member Type,start_station_address,start_station_number,end_station_address,end_station_number
0,0,14h 26min. 2sec.,12/31/2010 23:49,1/1/2011 14:15,10th & U St NW (31111),10th & U St NW (31111),W00771,Casual,10th & U St NW,31111,10th & U St NW,31111.0
1,1,0h 8min. 34sec.,12/31/2010 23:37,12/31/2010 23:46,10th & U St NW (31111),14th & R St NW (31202),W01119,Registered,10th & U St NW,31111,14th & R St NW,31202.0
2,2,0h 12min. 17sec.,12/31/2010 23:27,12/31/2010 23:39,Park Rd & Holmead Pl NW (31602),14th St & Spring Rd NW (31401),W00973,Registered,Park Rd & Holmead Pl NW,31602,14th St & Spring Rd NW,31401.0
3,3,0h 15min. 53sec.,12/31/2010 23:21,12/31/2010 23:37,Calvert St & Woodley Pl NW (31106),14th St & Spring Rd NW (31401),W00914,Registered,Calvert St & Woodley Pl NW,31106,14th St & Spring Rd NW,31401.0
4,4,0h 36min. 19sec.,12/31/2010 23:20,12/31/2010 23:56,20th St & Florida Ave NW (31110),Columbus Circle / Union Station (31623),W00859,Casual,20th St & Florida Ave NW,31110,Columbus Circle / Union Station,31623.0


Now add columns for the latitude and longitude for the end stations.

Find another API that responds with JSON and provide a link. What data from it do you think would be interesting? How would you use it?